In [1]:
import pandas as pd
import numpy as np

In [35]:
ruta = '/Users/joseizammontt/Desktop/Universidad/master/thesis/codigo/DATA_UNV_2.CSV'
df = pd.read_csv(ruta, dtype={"COD_ZONAL": "string"})

In [36]:
#Seleccion de variables de interes
df = df[["ID_DIAANALISIS","DESC_LOCALFISICO","CLASE","COD_SKU","DESC_SKU","F_UNIDADESVENTA","F_MONTOVENTA","F_COSTOVENTA"]]
#Eliminamos transacciones que no son ventas
df = df[df.F_UNIDADESVENTA>0.000001]
df

,ID_DIAANALISIS,DESC_LOCALFISICO,CLASE,COD_SKU,DESC_SKU,F_UNIDADESVENTA,F_MONTOVENTA,F_COSTOVENTA
1,2021-11-03,Chiclayo 1,J03020101 - PESCADOS CONGELADOS,10163206,TRUCHA DESHUESADA CONGELADA XKG,0.960,30.84,25.436160
2,2021-10-31,Chiclayo 1,J03020101 - PESCADOS CONGELADOS,10163206,TRUCHA DESHUESADA CONGELADA XKG,0.464,15.72,12.294144
3,2021-10-21,Chiclayo 1,J03020101 - PESCADOS CONGELADOS,10163206,TRUCHA DESHUESADA CONGELADA XKG,1.502,37.52,39.783474
4,2021-11-10,Chiclayo 1,J03020101 - PESCADOS CONGELADOS,10163206,TRUCHA DESHUESADA CONGELADA XKG,0.408,13.83,10.810368
5,2021-11-09,Chiclayo 1,J03020101 - PESCADOS CONGELADOS,10163206,TRUCHA DESHUESADA CONGELADA XKG,1.250,40.70,33.120000
...,...,...,...,...,...,...,...,...
329967,2021-11-22,HB La Merced,J03010512 - PECHUGA,42464622,PECHUGA DE PAVITA EN TROZOS SF HB CONG X KG,0.510,7.85,6.321450
329968,2021-11-14,HB La Merced,J03010512 - PECHUGA,42464622,PECHUGA DE PAVITA EN TROZOS SF HB CONG X KG,1.650,25.41,20.460000
329969,2021-11-18,HB La Merced,J03010512 - PECHUGA,42464622,PECHUGA DE PAVITA EN TROZOS SF HB CONG X KG,0.475,7.31,5.887625
329970,2021-11-07,HB La Merced,J03010512 - PECHUGA,42464622,PECHUGA DE PAVITA EN TROZOS SF HB CONG X KG,1.090,16.78,13.519270


In [39]:
df['ID_DIAANALISIS'] = pd.to_datetime(df.ID_DIAANALISIS, format='%Y-%m-%d')
df['Week_Number'] = df['ID_DIAANALISIS'].dt.isocalendar().week

conditions = [
    (df['F_UNIDADESVENTA'] >= 1),
    (df['F_UNIDADESVENTA'] < 1)]
choices = [df['F_MONTOVENTA']/df['F_UNIDADESVENTA'], df['F_MONTOVENTA']]

df['MONTOVENTA_UNIDAD'] = np.select(conditions, choices)

conditions = [
    (df['F_UNIDADESVENTA'] >= 1),
    (df['F_UNIDADESVENTA'] < 1)]
choices = [df['F_COSTOVENTA']/df['F_UNIDADESVENTA'], df['F_COSTOVENTA']]

df['COSTOVENTA_UNIDAD'] = np.select(conditions, choices)

In [67]:
df_procesado = df.groupby(by=["Week_Number","DESC_LOCALFISICO","CLASE","COD_SKU", "DESC_SKU"]).agg(TOTAl_F_UNIDADESVENTA=('F_UNIDADESVENTA', 'sum'),
                                                                                                    TOTAl_F_MONTOVENTA=('F_MONTOVENTA', 'sum'),
                                                                                                    VENTA_PROMEDIO=('MONTOVENTA_UNIDAD', 'mean'),
                                                                                                    STD_VENTA_PROMEDIO=('MONTOVENTA_UNIDAD', 'std'),
                                                                                                    TOTAl_F_COSTOVENTA=('F_COSTOVENTA', 'sum'),
                                                                                                    COSTOVENTA_PROMEDIO=('COSTOVENTA_UNIDAD', 'mean'))
df_procesado = df_procesado.reset_index() 
df_procesado['COEF_VARIABILIDAD'] = df_procesado.STD_VENTA_PROMEDIO/df_procesado.VENTA_PROMEDIO
df_procesado.describe()

,Week_Number,COD_SKU,TOTAl_F_UNIDADESVENTA,TOTAl_F_MONTOVENTA,VENTA_PROMEDIO,STD_VENTA_PROMEDIO,TOTAl_F_COSTOVENTA,COSTOVENTA_PROMEDIO,COEF_VARIABILIDAD
count,82504.000000,8.250400e+04,82504.000000,82504.000000,82504.000000,67531.000000,82504.000000,82504.000000,67531.000000
mean,43.054834,3.903613e+07,34.848251,490.761325,21.041544,2.010354,368.591662,15.487560,0.093037
std,2.558357,7.609703e+06,189.210308,1775.714493,16.880902,3.824105,1470.216333,14.824134,0.143859
min,39.000000,1.015513e+07,0.002000,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000
25%,41.000000,4.042868e+07,3.000000,56.500000,10.896383,0.002068,40.913021,7.623066,0.000116
50%,43.000000,4.090551e+07,8.184000,159.540000,16.899823,0.383406,112.582028,11.943801,0.025249
75%,45.000000,4.187145e+07,22.648500,424.542500,25.411760,2.434183,299.912014,18.627276,0.121029
max,47.000000,4.256102e+07,11251.293000,104600.870000,239.899757,75.342589,85509.826800,266.670000,1.359332


In [68]:
df_procesado.head(5)

,Week_Number,DESC_LOCALFISICO,CLASE,COD_SKU,DESC_SKU,TOTAl_F_UNIDADESVENTA,TOTAl_F_MONTOVENTA,VENTA_PROMEDIO,STD_VENTA_PROMEDIO,TOTAl_F_COSTOVENTA,COSTOVENTA_PROMEDIO,COEF_VARIABILIDAD
0,39,Angamos,J03010101 - CARNE IMPORTADA,40688290,CARNE MOLIDA ANGUS TTN GRANEL X KG,19.130,804.44,41.899057,4.966648,2149.853,109.727579,0.118538
1,39,Angamos,J03010101 - CARNE IMPORTADA,40775224,COLITA DE CUADRIL PH X KG,5.235,487.71,98.905340,15.543140,357.540,68.298134,0.157152
2,39,Angamos,J03010101 - CARNE IMPORTADA,40775225,ENTRANA FINA PH X KG,8.170,1284.05,156.830723,18.472286,1032.295,126.351906,0.117785
3,39,Angamos,J03010101 - CARNE IMPORTADA,40950586,PICANHA PH X KG,1.945,242.92,121.147761,5.300135,174.768,87.159363,0.043749
4,39,Angamos,J03010101 - CARNE IMPORTADA,41125473,BIFE DE VACIO PH X KG,0.935,89.68,89.680000,NaN,75.861,75.861000,NaN


In [99]:
df_clases = pd.DataFrame(df_procesado.CLASE.value_counts()).reset_index()
df_clases = df_clases.rename({'index':'CLASE','CLASE':'frecuencia'}, axis=1)
df_clases

,CLASE,frecuencia
0,J03020101 - PESCADOS CONGELADOS,10288
1,J03020202 - MARISCOS CONGELADOS,5310
2,J03010410 - MENUDENCIAS,4255
3,J03020102 - FRESCO ENTERO,4016
4,J03010112 - PARRILLEROS,3904
...,...,...
74,J03010601 - ALPACA,1
75,J03010212 - ENTRECOT,1
76,J03010202 - BRAZUELO SIN HUESO (ASADO DE BRAZU...,1
77,J03010203 - BISTEC DE CERDO,1


In [276]:
clases = df_procesado.CLASE.unique()

In [317]:
for clase in clases:
    df_clase = df_procesado[df_procesado.CLASE==clase]
    SKUs = df_clase.COD_SKU.unique()
    weeks = df_clase.Week_Number.unique()
    locales = df_clase.DESC_LOCALFISICO.unique()

    etl_completa = pd.DataFrame()
    for local in locales:
        matriz_precios = np.zeros((np.shape(weeks)[0],np.shape(SKUs)[0]))

        for i in range(np.shape(weeks)[0]):
            for j in range(np.shape(SKUs)[0]):
                q = df_clase[(df_clase['DESC_LOCALFISICO']==local) & (df_clase['COD_SKU']==SKUs[j]) & (df_clase['Week_Number']==weeks[i])]
                r = np.array(q['VENTA_PROMEDIO'])
                if len(r)!=0:
                    matriz_precios[i,j] = r[0]
                else:
                    matriz_precios[i,j] = np.nan

        matriz_precios = pd.DataFrame(matriz_precios,columns = SKUs)
        
        matriz_precios["Week_Number"] = weeks
        #matriz_precios

        matriz_demanda_precios= pd.DataFrame()
        for i in range(np.shape(SKUs)[0]):
            demandas = np.zeros((1,np.shape(weeks)[0]))
            for j in range(np.shape(weeks)[0]):
                q = df_clase[(df_clase.DESC_LOCALFISICO==local)&(df_clase.COD_SKU==SKUs[i])& (df_clase.Week_Number==weeks[j])]
                r = np.array(q['VENTA_PROMEDIO'])
                if len(r)!=0:
                    demandas[0,j] = r[0]
                else:
                    demandas[0,j] = np.nan
             
            matriz_demanda_precios_aux = matriz_precios
            matriz_demanda_precios_aux['demanda'] = demandas[0]
            matriz_demanda_precios_aux['COD_SKU'] = SKUs[i]
            matriz_demanda_precios_aux['DESC_SKU'] = np.array(df_clase.DESC_SKU[df_clase.COD_SKU == SKUs[i]])[0]
            matriz_demanda_precios_aux['DESC_LOCALFISICO'] = local
            matriz_demanda_precios=pd.concat([matriz_demanda_precios,matriz_demanda_precios_aux])

        etl_completa = pd.concat([etl_completa,matriz_demanda_precios])
        narchivo = clase.replace('(','').replace(')','').replace('/','')
        etl_completa.to_csv(narchivo+'.csv',index=False)
        